<a href="https://colab.research.google.com/github/AkhdanFirdaus/bmn-model/blob/main/final_ta_klasifikasi_teks_bert_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Klasifikasi Teks Laporan Mobil Dinas dengan BERT

In [ ]:
import pandas as pd
import numpy as np

### Load Data

In [ ]:
# Mount Drive

from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
loc_master = '/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Dataset/Raw Data/raw_master_kendaraan.csv'
loc_invoice = '/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Dataset/Raw Data/raw_invoice.csv'
loc_sukucadang = '/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Dataset/Raw Data/raw_sukucadang.csv'
loc_kerusakan = '/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Dataset/Raw Data/raw_kategori_kerusakan.csv'
loc_laporan = '/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Dataset/Raw Data/raw_laporan.csv'

data_master_raw = pd.read_csv(loc_master)
data_invoice_raw = pd.read_csv(loc_invoice)
data_sukucadang_raw = pd.read_csv(loc_sukucadang)
data_kerusakan_raw = pd.read_csv(loc_kerusakan)
data_laporan_raw = pd.read_csv(loc_laporan)

# 4.3 Data Preparation

## 4.3.1 Pemilahan Data

### Perbaikan Data

## 4.3.2 Pembersihan Data

### Spell Checker

In [ ]:
# Spell Checker

### Casefolding

In [ ]:
def casefolding(val):
  return str(val).lower()

### Cleaning

In [ ]:
import re

def clearformat(val):
  # Membersihkan Whitespace
  val = re.sub(r'\s+', ' ', val)

  # Hanya Mengambil karakter alfanumerik
  val = re.sub("[^a-zA-Z0-9]", " ", val)

  return val

### Stemming

Hanya dilakuan pada
- data sukucadang = Keterangan Pekerjaan
- data kerusakan = Keterangan Kerusakan
- data level = keterangan level



In [ ]:
pip install Sastrawi PySastrawi

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

stemmer = StemmerFactory().create_stemmer()

def stemming(val):
  return stemmer.stem(str(val))

### Stopword Removal

Hanya dilakuan pada
- data sukucadang = Keterangan Pekerjaan
- data kerusakan = Keterangan Kerusakan
- data level = keterangan level

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

stopword_remover = StopWordRemoverFactory().create_stop_word_remover()

def stopwordremove(val):
  return stopword_remover.remove(str(val))

### Filtering Data

## 4.3.3 Konstruksi Data

### Penentuan Fitur

Akan dibuat sebuah dataset untuk mendeteksi konteks
1. menggabungkan data master dan invoice berdasar nomor polisi
2. menggabungkan data diatas dengan sukucadang berdasar pekerjaan

### Pemilihan Atribut

## 4.3.4 Pelabelan Data

### Penentuan Label

### Penerapan Label

## 4.3.5 Integrasi Data

### Penggabungan Data

### Pembentukan Dataset Akhir

### Balancing Data

### Split Dataset

# 4.4 Modeling

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Dataset/Dipilah/final_dataset3.csv', sep='\t')
LABELS = dataset.columns[1:]

In [ ]:
!pip install transformers tensorflow Sastrawi

In [ ]:
import tensorflow as tf
# tf.config.run_functions_eagerly(True)
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split

In [ ]:
# Prepare model variable and hyperparameters
MAX_LEN=256
TRAIN_BATCH_SIZE=32
TEST_BATCH_SIZE=32
EPOCHS=10
LEARNING_RATE=2e-05
TRAIN_SPLIT=0.8
TEST_SPLIT=0.2

In [ ]:
# Prepare Pre-trained Model dan Tokenizer
pretrainedbertdataset = 'indobenchmark/indobert-lite-base-p1'
bert_tokenizer = BertTokenizer.from_pretrained(pretrainedbertdataset)
bert_model = TFBertModel.from_pretrained(pretrainedbertdataset)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizerFast'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type albert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some layers from the model checkpoint at indobenchmark/indobert-lite-base-p1 were not used when initializing TFBertModel: ['predictions', 'albert', 'sop_classifier']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenc

In [ ]:
# Tokenizing Function
def tokenize_data(tokenizer, max_len, val):
  tokenized = tokenizer.encode_plus(
      val,
      add_special_tokens=True,
      max_length=max_len,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      return_token_type_ids=True,
      return_tensors='tf'
  )

  return tokenized

def tokenize_sentences(tokenizer, max_len, sentences):
  input_ids, attention_mask = [], []
  for sentence in sentences:
    tokenized = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='tf'
    )

    input_ids.append(tokenized['input_ids'])
    attention_mask.append(tokenized['attention_mask'])

  return {
    'input_ids': tf.convert_to_tensor(np.asarray(input_ids).squeeze(), dtype=tf.int32),
    'attention_mask': tf.convert_to_tensor(np.asarray(attention_mask).squeeze(), dtype=tf.int32)
  }

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

stemmer = StemmerFactory().create_stemmer()
stopword_remover = StopWordRemoverFactory().create_stop_word_remover()

def casefolding(val):
  return str(val).lower()

def stemming(val):
  return stemmer.stem(str(val))

def stopwordremove(val):
  return stopword_remover.remove(str(val))

def preprocess_sentences(tokenizer, max_len, sentences):
  input_ids, attention_mask = [], []
  for sentence in sentences:
    input = casefolding(sentence)
    input = stemming(input)
    input = stopwordremove(input)
    tokenized = tokenizer.encode_plus(
        input,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='tf'
    )

    input_ids.append(tokenized['input_ids'])
    attention_mask.append(tokenized['attention_mask'])

  return {
    'input_ids': tf.convert_to_tensor(np.asarray(input_ids).squeeze(), dtype=tf.int32),
    'attention_mask': tf.convert_to_tensor(np.asarray(attention_mask).squeeze(), dtype=tf.int32)
  }

In [ ]:
# Percobaan penggunaan tokenisasi
tokenized = tokenize_sentences(bert_tokenizer, MAX_LEN, [
    'Mobil saya tidak bisa di starter',
    'AC tidak dingin'
])

print(tokenized)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset['konteks'], dataset[LABELS], test_size=TEST_SPLIT, shuffle=True, random_state=42)
X_train = tokenize_sentences(bert_tokenizer, MAX_LEN, X_train)
X_test = tokenize_sentences(bert_tokenizer, MAX_LEN, X_test)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(TRAIN_BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(TEST_BATCH_SIZE)

In [ ]:
def classification_model(bert_encoder, num_labels, max_len, learning_rate):
  input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
  attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')

  encoding_layer = bert_encoder(input_ids, attention_mask)[0]

  l = tf.keras.layers.GlobalAveragePooling1D(name='pooling_layer')(encoding_layer)
  l = tf.keras.layers.Dropout(0.1, name='dropout_layer')(l)
  l = tf.keras.layers.Dense(num_labels, activation='sigmoid', name='output_layer')(l)

  model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=[l])

  OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  LOSS = tf.keras.losses.BinaryCrossentropy()
  METRICS = [tf.keras.metrics.BinaryAccuracy(name='accuracy')]

  model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)

  return model

model = classification_model(
    bert_encoder=bert_model,
    num_labels=len(LABELS),
    max_len=MAX_LEN,
    learning_rate=LEARNING_RATE,
)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109081344   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:
model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset,
)

Epoch 1/10


8/8 [==============================] - 74s 2s/step - loss: 0.5048 - accuracy: 0.8147 - val_loss: 0.3267 - val_accuracy: 0.8968
Epoch 2/10
8/8 [==============================] - 12s 2s/step - loss: 0.3396 - accuracy: 0.8814 - val_loss: 0.2826 - val_accuracy: 0.8906
Epoch 3/10
8/8 [==============================] - 12s 2s/step - loss: 0.2812 - accuracy: 0.8892 - val_loss: 0.2567 - val_accuracy: 0.9045
Epoch 4/10
8/8 [==============================] - 12s 2s/step - loss: 0.2294 - accuracy: 0.9032 - val_loss: 0.2111 - val_accuracy: 0.9384
Epoch 5/10
8/8 [==============================] - 13s 2s/step - loss: 0.1862 - accuracy: 0.9263 - val_loss: 0.1935 - val_accuracy: 0.9353
Epoch 6/10
8/8 [==============================] - 13s 2s/step - loss: 0.1512 - accuracy: 0.9504 - val_loss: 0.1617 - val_accuracy: 0.9553
Epoch 7/10
8/8 [==============================] - 13s 2s/step - loss: 0.1283 - accuracy: 0.9493 - val_loss: 0.1368 - val_accuracy: 0.9538
Epoch 8/10
8/8 [=============================

# 4.5 Model Evaluation

In [ ]:
model.evaluate(test_dataset)

2/2 [==============================] - 1s 504ms/step - loss: 0.1187 - accuracy: 0.9646


[0.11871889233589172, 0.964560866355896]

In [ ]:
inputs = ['AC mobil tidak dingin', 'Rem pada mobil tidak bekerja', 'Kaca spion rusak harus diganti']
tokenized = preprocess_sentences(bert_tokenizer, MAX_LEN, inputs)
predictions = model.predict(tokenized)

# Define the threshold
threshold = 0.5

# Apply the threshold to the predictions
binary_predictions = np.where(predictions > threshold, 1, 0)

# Print the binary predictions
print(LABELS)
print(binary_predictions)

1/1 [==============================] - 7s 7s/step
Index(['masalah sistem knalpot', 'masalah transmisi', 'masalah suspensi',
       'gangguan listrik', 'masalah sistem bahan bakar', 'masalah kemudi',
       'masalah ban', 'masalah mesin', 'kegagalan rem',
       'masalah sistem pendingin', 'kerusakan aksesoris interior eksterior'],
      dtype='object')
[[0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]]


In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming you have predictions and ground truth labels
predictions = model.predict(test_dataset)  # Get model predictions
threshold = 0.5  # Set threshold for label prediction

# Convert probability predictions to binary predictions
binary_predictions = np.where(predictions >= threshold, 1, 0)

# Compute precision, recall, and F1-score for each label
label_precisions = precision_score(y_test, binary_predictions, average=None)
label_recalls = recall_score(y_test, binary_predictions, average=None)
label_f1_scores = f1_score(y_test, binary_predictions, average=None)

# Print accuracy metrics for each label
for label, precision, recall, f1_score in zip(LABELS, label_precisions, label_recalls, label_f1_scores):
    print(f"Label: {label}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1_score}")
    print()


2/2 [==============================] - 5s 468ms/step
Label: masalah sistem knalpot
Precision: 1.0
Recall: 0.8333333333333334
F1-score: 0.9090909090909091

Label: masalah transmisi
Precision: 0.8888888888888888
Recall: 1.0
F1-score: 0.9411764705882353

Label: masalah suspensi
Precision: 0.3333333333333333
Recall: 0.3333333333333333
F1-score: 0.3333333333333333

Label: gangguan listrik
Precision: 0.875
Recall: 1.0
F1-score: 0.9333333333333333

Label: masalah sistem bahan bakar
Precision: 1.0
Recall: 0.6666666666666666
F1-score: 0.8

Label: masalah kemudi
Precision: 0.7142857142857143
Recall: 1.0
F1-score: 0.8333333333333333

Label: masalah ban
Precision: 0.75
Recall: 0.8571428571428571
F1-score: 0.7999999999999999

Label: masalah mesin
Precision: 1.0
Recall: 0.3333333333333333
F1-score: 0.5

Label: kegagalan rem
Precision: 1.0
Recall: 1.0
F1-score: 1.0

Label: masalah sistem pendingin
Precision: 0.8
Recall: 1.0
F1-score: 0.888888888888889

Label: kerusakan aksesoris interior eksterior
Pr

# 4.6 Deployment

In [ ]:
!rm model_output/klasifikasi.h5
model.save('model_output/klasifikasi.h5')

rm: cannot remove 'model_output/klasifikasi2.h5': No such file or directory


In [ ]:
loaded_model = tf.keras.models.load_model('/content/model_output/klasifikasi.h5', custom_objects={'TFBertModel': TFBertModel})

In [ ]:
inputs = ['mesin tidak bisa menyala', 'AC mobil tidak dingin', 'Mesin bersuara kasar', 'ban bocor dan harus diganti dengan yang baru', 'asap knalpot berwarna putih', 'kaca spion pecah dan harus diganti', 'lampu sein kendaraan tidak berfungsi']
tokenized = preprocess_sentences(bert_tokenizer, MAX_LEN, inputs)
predictions = loaded_model.predict(tokenized)

# Define the threshold
threshold = 0.5

# Apply the threshold to the predictions
binary_predictions = np.where(predictions > threshold, 1, 0)

# Print the binary predictions
print(LABELS)
print(binary_predictions)

1/1 [==============================] - 6s 6s/step
Index(['masalah sistem knalpot', 'masalah transmisi', 'masalah suspensi',
       'gangguan listrik', 'masalah sistem bahan bakar', 'masalah kemudi',
       'masalah ban', 'masalah mesin', 'kegagalan rem',
       'masalah sistem pendingin', 'kerusakan aksesoris interior eksterior'],
      dtype='object')
[[0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0]]


## Implementasi Rumus menentukan tingkat kerusakan

In [ ]:
LABEL_WEIGHTS = dict.fromkeys(label_kerusakan, 0)

def measure_severity(val):
  treshold = 0.5

  NUMBER_LABEL_DETECTED = np.sum(val > treshold)
  LABEL_RESULTS = {}

  for label, count in zip(LABEL_WEIGHTS.keys(), val):
    if count > treshold:
      LABEL_RESULTS[label] = count

  category_weights = [LABEL_WEIGHTS[label] * count for label, count in LABEL_RESULTS.items()]
  severity_score = sum(category_weights) / NUMBER_LABEL_DETECTED

  return severity_score